# Model South Korea Time Series for COVID: Publish for app

This notebook use multistep time serie model  (predicting number of cases future next days).

It converts Tensorflow model into TensorFlow Lite to be able to use it in a Lambda fonction on AWS.

After that, the lite model is tested and publish on AWS

Finally, the lambda function is tested

## Convert model in TFlite

**IMPORTANT TO DO manually:**
- **Update TRAIN_SPLIT in my_helpers.model module**

### import

In [1]:
# data useful lib
import pandas as pd
import numpy as np

# helper lib
import shutil
import os, stat
import re
import datetime
import math

# read json from http
import json
import urllib.request

# read csv from http
import io
import requests

# model lib
import tensorflow as tf

# from project
from my_helpers.data_plots_kr import load_df_feat_kr
from my_helpers.model import prepare_to_lambda, retrieve_from_lambda
from my_helpers.model import prepare_to_lambda_future
from my_helpers.model import create_list_past_hist, predict_list

from my_helpers.model_kr import prepare_data_features_kr
from my_helpers.model_kr import prepare_dataset_kr

from my_helpers.utils import clean_file

import settings

In [2]:
tf.__version__

'2.5.0'

### Definitions

In [3]:
PATH_TO_SAVE_DATA = settings.PATH_TO_SAVE_DATA

from my_helpers.model_kr import PATH_MDL_MULTI_STEP_KR
from my_helpers.model_kr import PATH_MDL_MULTI_TFLITE_KR 
from my_helpers.model_kr import PATH_MDL_MULTI_TFLITE_FILE_KR
from my_helpers.model_kr import PATH_SERVERLESS_KR
from my_helpers.model_kr import URL_PREDICT_KR
from my_helpers.model_kr import PAST_HISTORY # days used to predict next values in future
from my_helpers.model_kr import FUTURE_TARGET  # predict 3 days later
from my_helpers.model_kr import STEP 

from my_helpers.data_plots_kr import PATH_DF_FEAT_KR

date_format = "%Y-%m-%d"

#NB_POS_DATE_MIN_DF_FEAT = 140227 # on 12/05/2020

# plot
NB_DAY_PLOT = FUTURE_TARGET*9

# train split
from my_helpers.model_kr import TRAIN_SPLIT
print(f"TRAIN_SPLIT = {TRAIN_SPLIT}")


TRAIN_SPLIT = 439


### helper functions

### load data

In [4]:
# load
df_feat_kr = load_df_feat_kr()
# clean
df_feat_kr = prepare_data_features_kr(df_feat_kr)
# split train/test
df_feat_kr["train"] = [True if I <= TRAIN_SPLIT else False \
                       for I in range(df_feat_kr.shape[0])]
df_feat_kr

,nb_cases,nb_tests,nb_deaths,date,day_num,Jeju,Gyeongnam,Gyeongbuk,Jeonnam,Jeonbuk,...,Rt_Jeonbuk,sum_Jeonnam,Rt_Jeonnam,sum_Gyeongbuk,Rt_Gyeongbuk,sum_Gyeongnam,Rt_Gyeongnam,sum_Jeju,Rt_Jeju,train
date,,,,,,,,,,,,,,,,,,,,,
2020-04-03,10062.0,424365.0,174.0,2020-04-03,5.0,0.0,1.0,5.0,1.0,1.0,...,5.000000,10.0,inf,106.0,4.240000,20.0,20.000000,5.0,inf,True
2020-04-04,10156.0,434888.0,177.0,2020-04-04,6.0,0.0,1.0,1.0,0.0,0.0,...,5.000000,10.0,inf,67.0,1.030769,21.0,21.000000,5.0,inf,True
2020-04-05,10237.0,441662.0,183.0,2020-04-05,0.0,3.0,1.0,4.0,0.0,1.0,...,6.000000,9.0,9.000000,60.0,0.789474,21.0,10.500000,8.0,inf,True
2020-04-06,10284.0,447509.0,186.0,2020-04-06,1.0,0.0,2.0,2.0,0.0,0.0,...,6.000000,9.0,9.000000,60.0,0.769231,22.0,7.333333,8.0,inf,True
2020-04-07,10331.0,456654.0,192.0,2020-04-07,2.0,0.0,1.0,1.0,0.0,0.0,...,6.000000,9.0,9.000000,60.0,0.759494,22.0,5.500000,8.0,inf,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-29,308723.0,13347641.0,2474.0,2021-09-29,3.0,5.0,84.0,88.0,23.0,41.0,...,2.096429,218.0,0.968889,672.0,1.476923,658.0,0.937322,117.0,0.893130,False
2021-09-30,311286.0,13394597.0,2481.0,2021-09-30,4.0,15.0,67.0,52.0,24.0,35.0,...,2.134752,234.0,1.063636,691.0,1.549327,692.0,1.016153,121.0,0.923664,False
2021-10-01,313772.0,13427198.0,2497.0,2021-10-01,5.0,9.0,69.0,60.0,29.0,32.0,...,2.062069,250.0,1.146789,713.0,1.627854,726.0,1.091729,121.0,0.930769,False


In [5]:
df_feat_kr[df_feat_kr["train"] == False].head(5)

,nb_cases,nb_tests,nb_deaths,date,day_num,Jeju,Gyeongnam,Gyeongbuk,Jeonnam,Jeonbuk,...,Rt_Jeonbuk,sum_Jeonnam,Rt_Jeonnam,sum_Gyeongbuk,Rt_Gyeongbuk,sum_Gyeongnam,Rt_Gyeongnam,sum_Jeju,Rt_Jeju,train
date,,,,,,,,,,,,,,,,,,,,,
2021-06-17,149731.0,10136517.0,1994.0,2021-06-17,4.0,8.0,5.0,3.0,6.0,7.0,...,0.461538,89.0,0.654412,104.0,0.611765,310.0,1.270492,147.0,0.913043,False
2021-06-18,150238.0,10168211.0,1996.0,2021-06-18,5.0,6.0,14.0,11.0,4.0,4.0,...,0.528846,81.0,0.618321,104.0,0.611765,280.0,1.037037,132.0,0.745763,False
2021-06-19,150720.0,10194433.0,1997.0,2021-06-19,6.0,7.0,11.0,4.0,5.0,5.0,...,0.433962,72.0,0.558140,98.0,0.597561,271.0,1.007435,117.0,0.609375,False
2021-06-20,151149.0,10204572.0,2002.0,2021-06-20,0.0,2.0,11.0,6.0,3.0,3.0,...,0.532609,66.0,0.515625,94.0,0.580247,265.0,0.985130,100.0,0.492611,False
2021-06-21,151506.0,10217614.0,2004.0,2021-06-21,1.0,3.0,13.0,8.0,10.0,3.0,...,0.538462,73.0,0.623932,93.0,0.584906,250.0,0.892857,90.0,0.456853,False


### Prepare dataset

In [6]:
dataset, data_std, data_mean = prepare_dataset_kr(df_feat_kr)
print(dataset)
print("data_mean : ", data_mean)
print("data_std : ", data_std)

[[-0.52130673 -0.1031765  -1.10889485 ... -0.70980736  0.99915011
  -0.34245495]
 [-0.5177716  -0.19956414 -1.28111519 ... -0.70881939  1.49815682
  -0.34245495]
 [-0.7970468  -0.57740371 -1.79588107 ... -0.94031407 -1.49588344
  -0.34245495]
 ...
 [ 0.63821562  0.98022063  0.93216493 ...  0.6544614   0.99915011
  -1.25058298]
 [ 0.76548027  0.98793164  0.74857378 ...  0.98598847  1.49815682
  -1.27792609]
 [ 0.69831282  0.48286038  1.61749291 ... -0.92135732 -1.49588344
  -1.27792609]]
data_mean :  [1.01488231e+01 1.97253607e+01 6.22433941e+01 1.86501139e+01
 3.15876993e+02 2.20021572e+04 2.99772210e+00 4.38250512e+01]
data_std :  [9.42916997e+00 8.64564472e+00 1.40711215e+01 6.00590525e+00
 3.16601549e+02 1.61947562e+04 2.00398107e+00 5.62668464e+00]


In [7]:
dataset.shape[1]

8

In [8]:
PAST_HISTORY

14

In [9]:
dataset.shape[1]

8

### Load model

In [10]:
%%time
# reload best model
multi_step_model = tf.keras.models.load_model(PATH_MDL_MULTI_STEP_KR)

CPU times: user 1.34 s, sys: 53.7 ms, total: 1.4 s
Wall time: 1.45 s


In [11]:
multi_step_model.inputs[0].dtype

tf.float32

In [12]:
run_model = tf.function(lambda x: multi_step_model(x))
# This is important, let's fix the input size.
INPUT_SIZE = dataset.shape[1]
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([1, PAST_HISTORY, INPUT_SIZE],
                  multi_step_model.inputs[0].dtype))

# model directory.
MODEL_DIR = PATH_TO_SAVE_DATA + "/" + "keras_lstm"
multi_step_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)

'''converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.allow_custom_ops=True'''

tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/keras_lstm/assets


### Save model TFlite

In [13]:
clean_file(PATH_MDL_MULTI_TFLITE_FILE_KR)

File /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/converted_model_20211004_09_15_59.tflite moved!


In [14]:
open(PATH_MDL_MULTI_TFLITE_FILE_KR, "wb").write(tflite_model)

16684

## Test converted model

### Predict with TF model (not-converted one)

#### Past days

In [15]:
# TENSORFLOW MODEL :
# prepare list of past histories
list_x = create_list_past_hist(dataset)
# predict
y_multi_pred = predict_list(list_x, multi_step_model)
# convert in positive cases
y_pos_pred = (y_multi_pred * data_std[4]) + data_mean[4] 
y_pos_pred

[472 - 486]
[479 - 493]
[486 - 500]
[493 - 507]
[500 - 514]
[507 - 521]
[514 - 528]
[521 - 535]
[528 - 542]
9


array([[1260.4694, 1160.9229, 1165.4574, 1165.5531, 1154.5641, 1148.7303,
        1148.8328, 1272.7074, 1153.6986, 1158.1047, 1154.0541, 1141.6432,
        1134.2705, 1136.2869, 1233.2959, 1143.6906, 1150.3574, 1146.8824,
        1136.401 , 1129.2086, 1131.7054, 1314.1809, 1186.1752, 1190.2443,
        1187.3574, 1177.5183, 1169.4241, 1172.5819, 1291.4713, 1177.1617,
        1182.4962, 1178.139 , 1168.9706, 1160.4213, 1164.6143, 1170.6969,
        1097.6621, 1102.1625, 1101.6741, 1090.5089, 1085.5936, 1085.1849,
        1261.0684, 1151.3933, 1155.823 , 1151.1031, 1139.5338, 1130.4872,
        1134.6945, 1294.6132, 1175.636 , 1180.804 , 1176.0377, 1166.109 ,
        1156.1957, 1161.697 , 1314.4135, 1164.5364, 1167.9023, 1167.0577,
        1159.0155, 1151.7686, 1153.8883]], dtype=float32)

#### Future days

In [16]:
# prepare data : very last days
x_for_future = np.array([dataset[-PAST_HISTORY:,:]]) 
# predict next days
y_future_pred = multi_step_model.predict(x_for_future)

### Predict with TFlite & Compare 

#### Past days

In [17]:
# CONVERTED LITE MODEL
# load 
interpreter = tf.lite.Interpreter(model_content=tflite_model)

# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    # predict with tensorflow model
    expected = multi_step_model.predict(x_multi)
    # predict with TFlite model
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Reload Tlite model

In [18]:
interpreter = tf.lite.Interpreter(model_path=PATH_MDL_MULTI_TFLITE_FILE_KR)

In [19]:
PATH_MDL_MULTI_TFLITE_FILE_KR

'/Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/converted_model.tflite'

### Predict reloaded model

#### Past days

In [20]:
# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    
    expected = multi_step_model.predict(x_multi)
    
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


In [21]:
x_multi.shape

(1, 14, 8)

In [22]:
len(list_x)

9

## Update API lambda AWS

### API lambda simulate

#### Past days

In [23]:
dataset.shape

(549, 8)

In [24]:
json_list_list_x = prepare_to_lambda(dataset)
# simulate input to lambda (double dumps ? why ? i don't know yet)
json_list_list_x = json.dumps(json_list_list_x)
# simulate lambda

event = {"body": json_list_list_x}

[472 - 486]
[479 - 493]
[486 - 500]
[493 - 507]
[500 - 514]
[507 - 521]
[514 - 528]
[521 - 535]
[528 - 542]


In [25]:
# lambda code (file ./serverless/tensorflow-lite-on-aws-lambda/handler.py)
from serverless.tensorflow_lite_on_aws_lambda_kr.handler import predict
context = None
response = predict(event, context)


INPUT : nb. arr. : 9 / arrays shape: (1, 14, 8)
OUTPUT : nb. arrays : 9 / arrays shape in list: (1, 7)


In [26]:
# Retrieve from lambda in App code
# input : response
y_multi_pred_out = retrieve_from_lambda(response)      
y_multi_pred_out.shape

(1, 63)

In [27]:
y_multi_pred

array([[2.9835367, 2.6691148, 2.683437 , 2.6837394, 2.6490302, 2.6306043,
        2.6309276, 3.022191 , 2.6462967, 2.6602135, 2.6474192, 2.608219 ,
        2.584932 , 2.5913007, 2.8977082, 2.6146855, 2.6357434, 2.6247675,
        2.5916612, 2.568944 , 2.5768301, 3.153187 , 2.7488751, 2.7617278,
        2.7526093, 2.721532 , 2.695966 , 2.7059402, 3.0814576, 2.7204058,
        2.737255 , 2.7234929, 2.6945336, 2.6675305, 2.680774 , 2.6999865,
        2.4693027, 2.4835174, 2.4819748, 2.4467092, 2.431184 , 2.429893 ,
        2.9854286, 2.6390152, 2.6530066, 2.6380987, 2.6015563, 2.572982 ,
        2.5862713, 3.0913813, 2.715587 , 2.73191  , 2.7168558, 2.6854954,
        2.6541836, 2.6715598, 3.1539216, 2.6805282, 2.6911597, 2.6884918,
        2.6630902, 2.6402004, 2.6468956]], dtype=float32)

In [28]:
y_multi_pred_out

array([[2.98353624, 2.66911483, 2.68343711, 2.6837399 , 2.64902997,
        2.63060403, 2.63092709, 3.02219105, 2.64629626, 2.66021347,
        2.64741945, 2.60821891, 2.58493233, 2.59130096, 2.89770889,
        2.61468577, 2.63574386, 2.62476802, 2.59166145, 2.56894445,
        2.57683086, 3.15318727, 2.74887538, 2.76172757, 2.75260901,
        2.72153211, 2.69596601, 2.70594072, 3.08145761, 2.72040534,
        2.73725486, 2.72349286, 2.69453311, 2.66753078, 2.68077374,
        2.69998598, 2.46930242, 2.48351741, 2.4819746 , 2.44670868,
        2.43118358, 2.42989302, 2.98542833, 2.63901496, 2.65300679,
        2.63809896, 2.6015563 , 2.57298207, 2.58627129, 3.09138155,
        2.7155869 , 2.73191023, 2.716856  , 2.68549514, 2.65418363,
        2.67155981, 3.15392184, 2.68052793, 2.69115949, 2.68849206,
        2.66309023, 2.64020014, 2.64689565]])

In [29]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### Future days

In [30]:
# Prepare data to lambda (future)
json_list_list_x = prepare_to_lambda_future(dataset)

# simulate lambda
json_list_list_x = json.dumps(json_list_list_x) # dumps again : I dont know why
event = {"body": json_list_list_x}
context = None
response = predict(event, context)
y_future_pred_out = retrieve_from_lambda(response)      
y_future_pred_out.shape

INPUT : nb. arr. : 1 / arrays shape: (1, 14, 8)
OUTPUT : nb. arrays : 1 / arrays shape in list: (1, 7)


(1, 7)

In [31]:
y_future_pred_out

array([[3.14868689, 2.70926642, 2.72432876, 2.71968555, 2.69478297,
        2.67252994, 2.67944908]])

In [32]:
y_future_pred

array([[3.1486866, 2.7092664, 2.7243288, 2.7196858, 2.694783 , 2.67253  ,
        2.6794493]], dtype=float32)

In [33]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Update AWS Lambda with new model


This part does:
- Go to : ./serverless//tensorflow-lite-on-aws-lambda-kr
    
- Execute : sls deploy -v

In [34]:
str_exe = '#!/bin/bash\n' + \
    'export PATH="/usr/local/bin:$PATH"\n' + \
    f'cd {PATH_MDL_MULTI_TFLITE_KR}\n' + \
    'serverless deploy -v'
str_exe

'#!/bin/bash\nexport PATH="/usr/local/bin:$PATH"\ncd /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr\nserverless deploy -v'

In [35]:
open('deploy_serverless_kr.sh', "w").write(str_exe)
os.chmod('deploy_serverless_kr.sh', stat.S_IRWXU)

In [36]:
!cat ./deploy_serverless_kr.sh

#!/bin/bash
export PATH="/usr/local/bin:$PATH"
cd /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr
serverless deploy -v

In [37]:
!./deploy_serverless_kr.sh

Serverless: Deprecation warning: Resolution of lambda version hashes was improved with better algorithm, which will be used in next major release.
            Switch to it now by setting "provider.lambdaHashingVersion" to "20201221"
            More Info: https://www.serverless.com/framework/docs/deprecations/#LAMBDA_HASHING_VERSION_V2
Serverless: Generated requirements from /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/requirements.txt in /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/.serverless/requirements.txt...
Serverless: Using static cache of requirements found at /Users/gregory/Library/Caches/serverless-python-requirements/eddadf68f9f2c21cf494701082a5393692d7b3c4711137913b3ed1b03bec68cf_slspyc ...
Serverless: Packaging service...
Serverless: Excluding development dependencies...
Serverle

### API AWS real Test

#### Past days

In [38]:
# prepare input
json_list_list_x = prepare_to_lambda(dataset)
# REQUEST
resp = requests.post(URL_PREDICT_KR, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

[472 - 486]
[479 - 493]
[486 - 500]
[493 - 507]
[500 - 514]
[507 - 521]
[514 - 528]
[521 - 535]
[528 - 542]
status code :  200
[[[2.9835362434387207, 2.669114828109741, 2.6834371089935303, 2.6837399005889893, 2.6490299701690674, 2.6306040287017822, 2.630927085876465]], [[3.022191047668457, 2.646296262741089, 2.6602134704589844, 2.6474194526672363, 2.6082189083099365, 2.584932327270508, 2.5913009643554688]], [[2.8977088928222656, 2.6146857738494873, 2.6357438564300537, 2.624768018722534, 2.5916614532470703, 2.5689444541931152, 2.5768308639526367]], [[3.1531872749328613, 2.748875379562378, 2.7617275714874268, 2.7526090145111084, 2.721532106399536, 2.6959660053253174, 2.7059407234191895]], [[3.0814576148986816, 2.720405340194702, 2.7372548580169678, 2.7234928607940674, 2.694533109664917, 2.6675307750701904, 2.6807737350463867]], [[2.699985980987549, 2.4693024158477783, 2.4835174083709717, 2.4819746017456055, 2.4467086791992188, 2.4311835765838623, 2.4298930168151855]], [[2.985428333282470

In [39]:
len(json_list_list_x)

20625

In [40]:
resp.json()

[[[2.9835362434387207,
   2.669114828109741,
   2.6834371089935303,
   2.6837399005889893,
   2.6490299701690674,
   2.6306040287017822,
   2.630927085876465]],
 [[3.022191047668457,
   2.646296262741089,
   2.6602134704589844,
   2.6474194526672363,
   2.6082189083099365,
   2.584932327270508,
   2.5913009643554688]],
 [[2.8977088928222656,
   2.6146857738494873,
   2.6357438564300537,
   2.624768018722534,
   2.5916614532470703,
   2.5689444541931152,
   2.5768308639526367]],
 [[3.1531872749328613,
   2.748875379562378,
   2.7617275714874268,
   2.7526090145111084,
   2.721532106399536,
   2.6959660053253174,
   2.7059407234191895]],
 [[3.0814576148986816,
   2.720405340194702,
   2.7372548580169678,
   2.7234928607940674,
   2.694533109664917,
   2.6675307750701904,
   2.6807737350463867]],
 [[2.699985980987549,
   2.4693024158477783,
   2.4835174083709717,
   2.4819746017456055,
   2.4467086791992188,
   2.4311835765838623,
   2.4298930168151855]],
 [[2.9854283332824707,
   2.63901

In [41]:
y_multi_pred_out = retrieve_from_lambda(resp)      
y_multi_pred_out.shape

(1, 63)

In [42]:
y_multi_pred_out

array([[2.98353624, 2.66911483, 2.68343711, 2.6837399 , 2.64902997,
        2.63060403, 2.63092709, 3.02219105, 2.64629626, 2.66021347,
        2.64741945, 2.60821891, 2.58493233, 2.59130096, 2.89770889,
        2.61468577, 2.63574386, 2.62476802, 2.59166145, 2.56894445,
        2.57683086, 3.15318727, 2.74887538, 2.76172757, 2.75260901,
        2.72153211, 2.69596601, 2.70594072, 3.08145761, 2.72040534,
        2.73725486, 2.72349286, 2.69453311, 2.66753078, 2.68077374,
        2.69998598, 2.46930242, 2.48351741, 2.4819746 , 2.44670868,
        2.43118358, 2.42989302, 2.98542833, 2.63901496, 2.65300679,
        2.63809896, 2.6015563 , 2.57298207, 2.58627129, 3.09138155,
        2.7155869 , 2.73191023, 2.716856  , 2.68549514, 2.65418363,
        2.67155981, 3.15392184, 2.68052793, 2.69115949, 2.68849206,
        2.66309023, 2.64020014, 2.64689565]])

In [43]:
y_multi_pred

array([[2.9835367, 2.6691148, 2.683437 , 2.6837394, 2.6490302, 2.6306043,
        2.6309276, 3.022191 , 2.6462967, 2.6602135, 2.6474192, 2.608219 ,
        2.584932 , 2.5913007, 2.8977082, 2.6146855, 2.6357434, 2.6247675,
        2.5916612, 2.568944 , 2.5768301, 3.153187 , 2.7488751, 2.7617278,
        2.7526093, 2.721532 , 2.695966 , 2.7059402, 3.0814576, 2.7204058,
        2.737255 , 2.7234929, 2.6945336, 2.6675305, 2.680774 , 2.6999865,
        2.4693027, 2.4835174, 2.4819748, 2.4467092, 2.431184 , 2.429893 ,
        2.9854286, 2.6390152, 2.6530066, 2.6380987, 2.6015563, 2.572982 ,
        2.5862713, 3.0913813, 2.715587 , 2.73191  , 2.7168558, 2.6854954,
        2.6541836, 2.6715598, 3.1539216, 2.6805282, 2.6911597, 2.6884918,
        2.6630902, 2.6402004, 2.6468956]], dtype=float32)

In [44]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### future days

In [45]:
# prepare input
json_list_list_x = prepare_to_lambda_future(dataset)
resp = requests.post(URL_PREDICT_KR, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

status code :  200
[[[3.1486868858337402, 2.709266424179077, 2.7243287563323975, 2.7196855545043945, 2.6947829723358154, 2.672529935836792, 2.6794490814208984]]]


In [46]:
y_future_pred_out = retrieve_from_lambda(resp)      
y_future_pred_out

array([[3.14868689, 2.70926642, 2.72432876, 2.71968555, 2.69478297,
        2.67252994, 2.67944908]])

In [47]:
y_future_pred

array([[3.1486866, 2.7092664, 2.7243288, 2.7196858, 2.694783 , 2.67253  ,
        2.6794493]], dtype=float32)

In [48]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.
